# Analysis of poweroutages

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

In [323]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

## Step 2: Data Cleaning and Exploratory Data Analysis

### Data cleaning

In [324]:
data = pd.read_excel('data/outage.xlsx', skiprows=5)

In [325]:
data = data.drop(columns = 'variables')
data = data[1:]

In [326]:
data.set_index('OBS', inplace=True)

In [327]:
columns_to_keep = [
    'YEAR',
    'MONTH',
    'U.S._STATE',
    'NERC.REGION',
    'ANOMALY.LEVEL',
    'CLIMATE.CATEGORY',
    'OUTAGE.START.DATE',
    'OUTAGE.START.TIME',
    'OUTAGE.RESTORATION.DATE',
    'OUTAGE.RESTORATION.TIME',
    'CAUSE.CATEGORY',
    'CAUSE.CATEGORY.DETAIL',
    'OUTAGE.DURATION',
    'DEMAND.LOSS.MW',
    'CUSTOMERS.AFFECTED',
    'RES.CUSTOMERS',
    'COM.CUSTOMERS',
    'IND.CUSTOMERS',
    'TOTAL.CUSTOMERS'
]

Dropping irrelevant rows

In [328]:
data = data[columns_to_keep]

Combining Outage.start.date and outage.start.time, same thing with outage.restoration

In [329]:
# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME' into a single 'OUTAGE.START' column
data['OUTAGE.START'] = pd.to_datetime(data['OUTAGE.START.DATE']) + pd.to_timedelta(data['OUTAGE.START.TIME'].astype(str))

# Combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME' into a single 'OUTAGE.RESTORATION' column
data['OUTAGE.RESTORATION'] = pd.to_datetime(data['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(data['OUTAGE.RESTORATION.TIME'].astype(str))

# Display the first few rows of the relevant columns to verify the results
data[['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.START', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'OUTAGE.RESTORATION']].head()

,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.START,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,OUTAGE.RESTORATION
OBS,,,,,,
1.0,2011-07-01 00:00:00,17:00:00,2011-07-01 17:00:00,2011-07-03 00:00:00,20:00:00,2011-07-03 20:00:00
2.0,2014-05-11 00:00:00,18:38:00,2014-05-11 18:38:00,2014-05-11 00:00:00,18:39:00,2014-05-11 18:39:00
3.0,2010-10-26 00:00:00,20:00:00,2010-10-26 20:00:00,2010-10-28 00:00:00,22:00:00,2010-10-28 22:00:00
4.0,2012-06-19 00:00:00,04:30:00,2012-06-19 04:30:00,2012-06-20 00:00:00,23:00:00,2012-06-20 23:00:00
5.0,2015-07-18 00:00:00,02:00:00,2015-07-18 02:00:00,2015-07-19 00:00:00,07:00:00,2015-07-19 07:00:00


In [330]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1534 entries, 1.0 to 1534.0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR                     1534 non-null   float64       
 1   MONTH                    1525 non-null   float64       
 2   U.S._STATE               1534 non-null   object        
 3   NERC.REGION              1534 non-null   object        
 4   ANOMALY.LEVEL            1525 non-null   object        
 5   CLIMATE.CATEGORY         1525 non-null   object        
 6   OUTAGE.START.DATE        1525 non-null   object        
 7   OUTAGE.START.TIME        1525 non-null   object        
 8   OUTAGE.RESTORATION.DATE  1476 non-null   object        
 9   OUTAGE.RESTORATION.TIME  1476 non-null   object        
 10  CAUSE.CATEGORY           1534 non-null   object        
 11  CAUSE.CATEGORY.DETAIL    1063 non-null   object        
 12  OUTAGE.DURATION          1476 non-n

## Exploratory Data Analysis

### Univariate Analysis

In [331]:
cause_counts = data['CAUSE.CATEGORY'].value_counts().reset_index()
cause_counts.columns = ['CAUSE.CATEGORY', 'count']


In [332]:
fig = px.bar(
    cause_counts,
    x='CAUSE.CATEGORY',
    y='count',
    title='Count of Outages by Cause Category',
    labels={'CAUSE.CATEGORY': 'Cause Category', 'count': 'Number of Outages'}
)

# Show the plot
fig.show()

In [333]:
# Count the occurrences of each U.S. state in 'U.S._STATE'
state_counts = data['U.S._STATE'].value_counts().reset_index()

# Rename the columns for clarity
state_counts.columns = ['U.S._STATE', 'count']
# Create a bar plot
fig = px.bar(
    state_counts,
    x='U.S._STATE',
    y='count',
    title='Count of Outages by U.S. State',
    labels={'U.S._STATE': 'U.S. State', 'count': 'Number of Outages'}
)

# Show the plot
fig.show()

## Bivariate analysis

Purpose: Understand how outage duration varies by the cause of the outage. This helps identify if certain causes (like severe weather) are linked to longer outages.

In [334]:
fig = px.box(
    data,
    x='CAUSE.CATEGORY',
    y='CUSTOMERS.AFFECTED',
    title='Customers Affected by Cause Category',
    labels={'CAUSE.CATEGORY': 'Cause Category', 'CUSTOMERS.AFFECTED': 'Customers Affected'}
)

fig.show()

In [335]:
import folium
import geopandas as gpd
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim  # Import Nominatim from geopy
from folium.plugins import TimestampedGeoJson
import plotly.graph_objects as go

import time  # To add delays





Customer's affected by state

In [336]:
# URL to a GeoJSON file for U.S. state boundaries
url = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'

# Create a map centered on the U.S.
m = folium.Map(location=[37.8, -96], zoom_start=4)

# Aggregate total customers affected by state
state_customer_impact = data.groupby('U.S._STATE')['CUSTOMERS.AFFECTED'].sum().reset_index()

# Add a Choropleth layer to visualize the customers affected by state
folium.Choropleth(
    geo_data=url,
    data=state_customer_impact,
    columns=['U.S._STATE', 'CUSTOMERS.AFFECTED'],
    key_on='feature.properties.name',  # Matches GeoJSON 'name' property with 'U.S._STATE'
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Customers Affected by State'
).add_to(m)

# Display the map
m

Choropleth Map of Outage Duration by State
Purpose: Show which states experience the longest outages on average.



In [337]:
# URL to GeoJSON file for U.S. states
url = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'

# Calculate the average outage duration for each state
state_duration = data.groupby('U.S._STATE')['OUTAGE.DURATION'].mean().reset_index()

# Create the map centered on the U.S.
m = folium.Map(location=[37.8, -96], zoom_start=4)

# Add a Choropleth layer to visualize average outage duration
folium.Choropleth(
    geo_data=url,
    data=state_duration,
    columns=['U.S._STATE', 'OUTAGE.DURATION'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Outage Duration by State (hours)'
).add_to(m)

# Display the map
m

## Interesting Aggregates

Purpose: See which outage causes affect the most customers in each U.S. state.

In [338]:
pivot_customers = data.pivot_table(
    index='U.S._STATE',
    columns='CAUSE.CATEGORY',
    values='CUSTOMERS.AFFECTED',
    aggfunc='sum'
)

# View the table
display(pivot_customers)

CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
U.S._STATE,,,,,,,
Alabama,NaN,NaN,0.0,NaN,NaN,471644.0,NaN
Alaska,14273.0,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,167000.0,NaN,2713.0,NaN,NaN,180911.0,229000.0
Arkansas,0.0,NaN,9200.0,0.0,54094.0,556466.0,NaN
California,1390257.0,0.0,127920.0,131019.0,0.0,20579360.0,3344890.0
Colorado,NaN,0.0,0.0,35230.0,NaN,355058.0,61379.0
Connecticut,NaN,NaN,0.0,NaN,NaN,784410.0,NaN
Delaware,18400.0,NaN,0.0,NaN,NaN,65000.0,0.0
District of Columbia,52000.0,NaN,NaN,NaN,NaN,1700383.0,NaN


## Step 3: Assessment of Missingness

In [339]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1534 entries, 1.0 to 1534.0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR                     1534 non-null   float64       
 1   MONTH                    1525 non-null   float64       
 2   U.S._STATE               1534 non-null   object        
 3   NERC.REGION              1534 non-null   object        
 4   ANOMALY.LEVEL            1525 non-null   object        
 5   CLIMATE.CATEGORY         1525 non-null   object        
 6   OUTAGE.START.DATE        1525 non-null   object        
 7   OUTAGE.START.TIME        1525 non-null   object        
 8   OUTAGE.RESTORATION.DATE  1476 non-null   object        
 9   OUTAGE.RESTORATION.TIME  1476 non-null   object        
 10  CAUSE.CATEGORY           1534 non-null   object        
 11  CAUSE.CATEGORY.DETAIL    1063 non-null   object        
 12  OUTAGE.DURATION          1476 non-n

## NMAR

NMAR: DEMAND.LOSS.MW: This column indicates the megawatt demand loss during an outage. The missingness could be NMAR if the probability of missing data depends on the magnitude of the demand loss itself. For example, minor outages might not have detailed demand loss recorded, leading to missing values. Alternatively, in significant outages, the complexity of measuring demand loss might result in missing data.

## Missingness Dependancy

In [340]:
import plotly.figure_factory as ff

In [341]:
missing_data = data.copy()

In [342]:
missing_data['DEMAND.LOSS.MW_missing'] = missing_data['DEMAND.LOSS.MW'].isnull()

In [343]:
# Used for plotting examples.
def create_kde_plotly(df, group_col, group1, group2, vals_col, title=''):
    fig = ff.create_distplot(
        hist_data=[df.loc[df[group_col] == group1, vals_col], df.loc[df[group_col] == group2, vals_col]],
        group_labels=[group1, group2],
        show_rug=False, show_hist=False,
        colors=['#ef553b', '#636efb'],
    )
    return fig.update_layout(title=title)

In [344]:
def mcarmar(data, missing_column='DEMAND.LOSS.MW', column='NERC.REGION', n_repetitions=500):
    """
    Test if missingness in a specified column is MCAR or MAR with respect to another column in the dataset.
    
    Parameters:
    - data: DataFrame containing the data to be tested.
    - missing_column: Name of the column to check for missingness.
    - column: Name of the column to group by.
    - n_repetitions: Number of random permutations for the TVD calculation.
    
    Returns:
    - A dictionary containing observed TVD, p-value, and the conclusion (MCAR or MAR).
    """
    # Step 1: Create the binary "missingness" column for the specified missing column
    demand_missing = (
        data
        .assign(missing_indicator=data[missing_column].isna())
        .pivot_table(index=column, columns='missing_indicator', aggfunc='size')
    )

    # Ensure columns are named clearly
    demand_missing.columns = ['missing = False', 'missing = True']
    demand_missing = demand_missing / demand_missing.sum()

    # Visualize the proportion of missing data for each category of the grouping column
    display(demand_missing.plot(kind='barh', title=f'{column} by Missingness of {missing_column}', barmode='group'))

    # Step 2: Calculate observed TVD
    observed_tvd = demand_missing.diff(axis=1).iloc[:, -1].abs().sum() / 2
    
    tvds = []
    for _ in range(n_repetitions):
        shuffled = data.copy()
        shuffled[column] = np.random.permutation(shuffled[column])
        
        pivoted = (
            shuffled
            .assign(missing_indicator=shuffled[missing_column].isna())
            .pivot_table(index=column, columns='missing_indicator', aggfunc='size')
        )
        
        # Ensure missingness indicator columns are present, even if missing in the current permutation
        pivoted = pivoted.reindex(columns=[False, True], fill_value=0)
        pivoted = pivoted / pivoted.sum()
        
        tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
        tvds.append(tvd)
    
    # Calculate p-value
    p_value = np.mean(np.array(tvds) >= observed_tvd)
    
    # Plot the empirical distribution of the TVD
    fig = px.histogram(pd.DataFrame(tvds), x=0, nbins=50, histnorm='probability', 
                   title=f'Empirical Distribution of the TVD for {missing_column}')
    fig.add_vline(x=observed_tvd, line_color='red', line_width=2, opacity=1)
    fig.add_annotation(
        text=f'<span style="color:red">Observed TVD = {round(observed_tvd, 4)}</span>',
        x=observed_tvd * 1.1, 
        y=0.15, 
        showarrow=False
    )
    fig.update_layout(yaxis_range=[0, 0.2])
    fig.show()

    # Print results
    print(f'Observed TVD: {observed_tvd:.4f}, p-value: {p_value:.4f}')
    print(f'MCAR vs MAR conclusion: {"MAR" if p_value < 0.01 else "MCAR"}')
    
    return {
        'observed_tvd': observed_tvd,
        'p_value': p_value,
        'conclusion': 'MAR' if p_value < 0.01 else 'MCAR'
    }


### Checking if missingness of demand loss is dependant on month

In [345]:
mcarmar(missing_data, 'DEMAND.LOSS.MW', 'MONTH', 5000 )

Observed TVD: 0.1021, p-value: 0.0170
MCAR vs MAR conclusion: MCAR


{'observed_tvd': np.float64(0.1020811913886033),
 'p_value': np.float64(0.017),
 'conclusion': 'MCAR'}

### Checking if Demand Loss is missing at random on category of cause

In [346]:
mcarmar(missing_data,'DEMAND.LOSS.MW', 'CAUSE.CATEGORY', 10000)

Observed TVD: 0.1787, p-value: 0.0000
MCAR vs MAR conclusion: MAR


{'observed_tvd': np.float64(0.17869773888047633),
 'p_value': np.float64(0.0),
 'conclusion': 'MAR'}

Since we found that Demand.Loss.Mw is missing dependant on the cause category column we will now do a probabilistic imputation on demand loss based on the category.cause column

In [347]:
def prob_impute(s):
    s = s.copy()
    num_null = s.isna().sum()
    if num_null > 0 and len(s.dropna()) > 0:
        fill_values = np.random.choice(s.dropna(), num_null)
        s[s.isna()] = fill_values
    return s


In [348]:
data_data_filled = data.copy()
data_data_filled['DEMAND.LOSS.MW'] = (
    data_data_filled
    .groupby('CAUSE.CATEGORY')
    ['DEMAND.LOSS.MW']
    .transform(prob_impute)
)
data_data_filled['DEMAND.LOSS.MW'].head()

OBS
1.0    1655
2.0       0
3.0     125
4.0     176
5.0     250
Name: DEMAND.LOSS.MW, dtype: object

In [349]:
data_data_filled['DEMAND.LOSS.MW'].info()

<class 'pandas.core.series.Series'>
Index: 1534 entries, 1.0 to 1534.0
Series name: DEMAND.LOSS.MW
Non-Null Count  Dtype 
--------------  ----- 
1534 non-null   object
dtypes: object(1)
memory usage: 24.0+ KB


## Step 4: Hypothesis Testing

For our hypothesis test we'll be testing whether 

In [350]:
# TO

## Step 5: Framing a Prediction Problem

In [351]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1534 entries, 1.0 to 1534.0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR                     1534 non-null   float64       
 1   MONTH                    1525 non-null   float64       
 2   U.S._STATE               1534 non-null   object        
 3   NERC.REGION              1534 non-null   object        
 4   ANOMALY.LEVEL            1525 non-null   object        
 5   CLIMATE.CATEGORY         1525 non-null   object        
 6   OUTAGE.START.DATE        1525 non-null   object        
 7   OUTAGE.START.TIME        1525 non-null   object        
 8   OUTAGE.RESTORATION.DATE  1476 non-null   object        
 9   OUTAGE.RESTORATION.TIME  1476 non-null   object        
 10  CAUSE.CATEGORY           1534 non-null   object        
 11  CAUSE.CATEGORY.DETAIL    1063 non-null   object        
 12  OUTAGE.DURATION          1476 non-n

For our prediction problem we will be prediction the severity of a power outage interms of total customers affected

In [352]:
# TODO

## Step 6: Baseline Model

### Linear Regression

In [353]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [354]:
data.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'NERC.REGION', 'ANOMALY.LEVEL',
       'CLIMATE.CATEGORY', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME',
       'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY',
       'CAUSE.CATEGORY.DETAIL', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS',
       'TOTAL.CUSTOMERS', 'OUTAGE.START', 'OUTAGE.RESTORATION'],
      dtype='object')

In [355]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1534 entries, 1.0 to 1534.0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR                     1534 non-null   float64       
 1   MONTH                    1525 non-null   float64       
 2   U.S._STATE               1534 non-null   object        
 3   NERC.REGION              1534 non-null   object        
 4   ANOMALY.LEVEL            1525 non-null   object        
 5   CLIMATE.CATEGORY         1525 non-null   object        
 6   OUTAGE.START.DATE        1525 non-null   object        
 7   OUTAGE.START.TIME        1525 non-null   object        
 8   OUTAGE.RESTORATION.DATE  1476 non-null   object        
 9   OUTAGE.RESTORATION.TIME  1476 non-null   object        
 10  CAUSE.CATEGORY           1534 non-null   object        
 11  CAUSE.CATEGORY.DETAIL    1063 non-null   object        
 12  OUTAGE.DURATION          1476 non-n

For our baseline we will be using a random forest regressor to predict Total.Customers, we will use the demand.loss.mw and cause.category columns, we will also be standardizing demand.loss and using one hot encoding on cause.category

In [356]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [357]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Step 1: Prepare data
X = data_data_filled[['DEMAND.LOSS.MW', 'CAUSE.CATEGORY']]
y = data_data_filled['TOTAL.CUSTOMERS']

# Step 2: Clip or log-transform target
y = np.log1p(y)  # Apply log1p to target

# Step 3: Train-test split (no validation set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Step 4: Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('standardize', StandardScaler(), ['DEMAND.LOSS.MW']),
        ('cat', OneHotEncoder(), ['CAUSE.CATEGORY'])  # One-hot encode categorical features
    ]
)

# Step 5: Create pipeline with LinearRegression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),                               
    ('regressor', LinearRegression())
])

# Step 6: Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Step 7: Test set predictions
test_predictions = pipeline.predict(X_test)

# Step 8: Calculate MSE and RMSE
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)

# Step 9: Print results
print(f"Test MSE: {test_mse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")


Test MSE: 0.7966
Test RMSE: 0.8925


## Step 7: Final Model

In [358]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


In [359]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1534 entries, 1.0 to 1534.0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR                     1534 non-null   float64       
 1   MONTH                    1525 non-null   float64       
 2   U.S._STATE               1534 non-null   object        
 3   NERC.REGION              1534 non-null   object        
 4   ANOMALY.LEVEL            1525 non-null   object        
 5   CLIMATE.CATEGORY         1525 non-null   object        
 6   OUTAGE.START.DATE        1525 non-null   object        
 7   OUTAGE.START.TIME        1525 non-null   object        
 8   OUTAGE.RESTORATION.DATE  1476 non-null   object        
 9   OUTAGE.RESTORATION.TIME  1476 non-null   object        
 10  CAUSE.CATEGORY           1534 non-null   object        
 11  CAUSE.CATEGORY.DETAIL    1063 non-null   object        
 12  OUTAGE.DURATION          1476 non-n

In [362]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# 1. **Feature Selection**
# Select relevant columns (ignoring those with missing values and other specified columns)
X = data_data_filled[['YEAR', 'U.S._STATE', 'NERC.REGION', 'CAUSE.CATEGORY', 'DEMAND.LOSS.MW']]
y = data_data_filled['TOTAL.CUSTOMERS']
y = np.log1p(y)
# 2. **Train-Test Split**
# Split into 80% train and 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. **Preprocessor**
# Define column transformations with OneHotEncoder handling unknown categories
preprocessor = ColumnTransformer(
    transformers=[
        ('impute_num', SimpleImputer(strategy='median'), ['DEMAND.LOSS.MW']),  # Median imputation for missing numerical data
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'NERC.REGION', 'CAUSE.CATEGORY']),  # One-hot encoding for categorical columns
        ('ordinal', OrdinalEncoder(), ['YEAR'])  # Ordinal encoding for the YEAR column
    ]
)

# 4. **Pipeline**
# Combine preprocessor with a Random Forest regressor in a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# 5. **Hyperparameter Tuning**
# Define grid search parameters for hyperparameter optimization
param_grid = {
    'regressor__n_estimators': [100, 200],  # Number of trees
    'regressor__max_depth': [None, 10, 20],  # Maximum depth of trees
    'regressor__min_samples_split': [2, 5]  # Minimum samples to split an internal node
}

# Use GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 6. **Test the Model**
# Get the best model from GridSearchCV
best_model = grid_search.best_estimator_

# Test predictions and MSE
test_predictions = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
print(f"Test MSE: {test_mse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

# Display the best parameters
print("Best Parameters from GridSearchCV:", grid_search.best_params_)


Test MSE: 0.0138
Test RMSE: 0.1175
Best Parameters from GridSearchCV: {'regressor__max_depth': None, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 100}


In [369]:
fig = px.histogram(data, x='TOTAL.CUSTOMERS', nbins=50, title='Distribution of TOTAL.CUSTOMERS')
fig.update_layout(xaxis_title='Total Customers', yaxis_title='Count', showlegend=False)
fig.show()

np.float64(844.0)